In [1]:
from google.colab import drive
drive.mount('content/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at content/


In [2]:
ls

content/  sample_data/


In [3]:
cd 'content/My Drive/Dataset'

/content/content/My Drive/Dataset


In [4]:
ls

my_model.h5  submission.csv  test/  test.csv  train/  train.csv


In [5]:
import numpy as np
import pandas as pd

In [6]:
training_set = pd.read_csv("train.csv")
training_imgs = [ x for x in list(training_set.Image) ]

In [7]:
print(training_imgs)

['96.jpg', '163.jpg', '450.jpg', '219.jpg', '455.jpg', '46.jpg', '326.jpg', '408.jpg', '365.jpg', '422.jpg', '397.jpg', '400.jpg', '32.jpg', '350.jpg', '234.jpg', '492.jpg', '333.jpg', '376.jpg', '103.jpg', '39.jpg', '87.jpg', '155.jpg', '388.jpg', '384.jpg', '248.jpg', '494.jpg', '47.jpg', '488.jpg', '462.jpg', '347.jpg', '242.jpg', '16.jpg', '178.jpg', '348.jpg', '164.jpg', '2.jpg', '141.jpg', '187.jpg', '464.jpg', '472.jpg', '489.jpg', '387.jpg', '109.jpg', '255.jpg', '274.jpg', '509.jpg', '19.jpg', '102.jpg', '24.jpg', '294.jpg', '501.jpg', '335.jpg', '321.jpg', '480.jpg', '393.jpg', '344.jpg', '259.jpg', '76.jpg', '343.jpg', '4.jpg', '486.jpg', '361.jpg', '90.jpg', '231.jpg', '331.jpg', '243.jpg', '104.jpg', '113.jpg', '21.jpg', '15.jpg', '370.jpg', '124.jpg', '241.jpg', '56.jpg', '296.jpg', '81.jpg', '29.jpg', '423.jpg', '445.jpg', '406.jpg', '116.jpg', '407.jpg', '118.jpg', '367.jpg', '30.jpg', '395.jpg', '265.jpg', '73.jpg', '277.jpg', '122.jpg', '516.jpg', '377.jpg', '513.jpg'

In [8]:
print(pd.value_counts(training_set.target))

mohiniyattam     50
odissi           49
kathakali        47
bharatanatyam    47
kuchipudi        46
sattriya         45
kathak           44
manipuri         36
Name: target, dtype: int64


In [9]:
training_set.target = pd.factorize(training_set.target)[0]
training_set.target = training_set.target.astype(str)
print(training_set.head(10))

     Image target
0   96.jpg      0
1  163.jpg      1
2  450.jpg      2
3  219.jpg      3
4  455.jpg      2
5   46.jpg      4
6  326.jpg      5
7  408.jpg      6
8  365.jpg      5
9  422.jpg      6


In [10]:
training_labels = list(training_set['target'])
training_set = pd.DataFrame( {'Image': training_imgs,'target': training_labels})

In [11]:
print(pd.value_counts(training_set.target))

7    50
2    49
3    47
1    47
6    46
5    45
4    44
0    36
Name: target, dtype: int64


In [12]:
print(training_set.head(5))

     Image target
0   96.jpg      0
1  163.jpg      1
2  450.jpg      2
3  219.jpg      3
4  455.jpg      2


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  horizontal_flip = True)

In [14]:
train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="train",x_col="Image",
                                        y_col="target",
                                        class_mode="categorical",
                                        target_size=(256,256),
                                        batch_size=32)

Found 364 validated image filenames belonging to 8 classes.


In [15]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D,GlobalAveragePooling2D,AveragePooling2D
from keras import applications
from keras import optimizers
from keras.applications import DenseNet169
from keras.models import Model

Using TensorFlow backend.


In [34]:
base_model = DenseNet169(weights='imagenet',include_top=False,input_shape=(256, 256, 3))

for layer in base_model.layers:
  layer.trainable = False

headModel = base_model.output
headModel = AveragePooling2D(4,4)(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(6656, activation="relu")(headModel)
headModel = Dense(8, activation="softmax")(headModel)

model = Model(inputs=base_model.input, outputs=headModel)

In [35]:
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['categorical_accuracy','accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_9 (ZeroPadding2D (None, 262, 262, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d_9[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1/conv[0][0]                 
____________________________________________________________________________________________

In [37]:
model.fit_generator(train_generator, epochs = 10, steps_per_epoch = len(training_set.Image)//32)

Epoch 1/10
11/11 [==============================] - 109s 10s/step - loss: 0.3026 - categorical_accuracy: 0.9148 - accuracy: 0.9148
Epoch 2/10
11/11 [==============================] - 98s 9s/step - loss: 0.2326 - categorical_accuracy: 0.9423 - accuracy: 0.9423
Epoch 3/10
11/11 [==============================] - 102s 9s/step - loss: 0.1629 - categorical_accuracy: 0.9578 - accuracy: 0.9578
Epoch 4/10
11/11 [==============================] - 109s 10s/step - loss: 0.0846 - categorical_accuracy: 0.9915 - accuracy: 0.9915
Epoch 5/10
11/11 [==============================] - 103s 9s/step - loss: 0.0841 - categorical_accuracy: 0.9880 - accuracy: 0.9880
Epoch 6/10
11/11 [==============================] - 103s 9s/step - loss: 0.0661 - categorical_accuracy: 0.9880 - accuracy: 0.9880
Epoch 7/10
11/11 [==============================] - 105s 10s/step - loss: 0.0335 - categorical_accuracy: 1.0000 - accuracy: 1.0000
Epoch 8/10
11/11 [==============================] - 96s 9s/step - loss: 0.0671 - categor

In [38]:
model.save("my_model.h5")

In [39]:
from keras.models import load_model
model = load_model("my_model.h5")

In [40]:
test_set = pd.read_csv("test.csv")
test_imgs = [x for x in list(test_set.Image)]
test_set = pd.DataFrame( {'Image': test_imgs })

In [41]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
    img = image.load_img(path="test/" + test_set.Image[i] ,target_size=(256,256,3))
    img = image.img_to_array(img)
    img = img/255.0
    test_img = img.reshape((1,256,256,3))
    img_class = model.predict(test_img)
    prediction = img_class[0]
    Y_pred.append(prediction)

In [42]:
print(Y_pred)

[array([2.2541759e-05, 7.6902643e-02, 7.7509612e-02, 3.5029338e-04,
       5.8788501e-02, 5.2644126e-03, 7.7433419e-01, 6.8277749e-03],
      dtype=float32), array([0.00797441, 0.03630055, 0.00325347, 0.00184934, 0.01387925,
       0.05603622, 0.00269962, 0.8780071 ], dtype=float32), array([7.7493882e-05, 2.3972771e-01, 4.8957819e-01, 3.6673864e-05,
       7.8540007e-03, 1.2896112e-01, 4.5270462e-02, 8.8494375e-02],
      dtype=float32), array([2.2854144e-04, 1.2974173e-01, 5.0695434e-02, 3.8571336e-04,
       1.2918056e-02, 6.0843863e-02, 7.3850501e-01, 6.6816644e-03],
      dtype=float32), array([2.1424019e-07, 9.9225265e-01, 1.3208375e-05, 1.2808266e-08,
       4.8344163e-04, 1.3168398e-04, 5.1880727e-04, 6.5999567e-03],
      dtype=float32), array([2.0112369e-05, 2.4703678e-01, 1.3520558e-03, 9.9408617e-06,
       1.4846999e-02, 6.6766720e-03, 7.2585922e-01, 4.1982555e-03],
      dtype=float32), array([6.5561239e-06, 9.5259643e-01, 2.3776207e-02, 4.2053111e-07,
       3.1513139e-03

In [43]:
predicted = []
for i in range(0, len(Y_pred)):
  max = 0
  k = 0
  for j in range(0, len(Y_pred[i])):
    if(Y_pred[i][j] >= max):
      max = Y_pred[i][j]
      k = j
  predicted.append(k)  

In [44]:
print(predicted)

[6, 7, 2, 6, 1, 6, 1, 3, 5, 1, 7, 1, 5, 5, 5, 1, 2, 4, 7, 1, 1, 3, 6, 2, 7, 3, 2, 1, 1, 1, 7, 6, 4, 1, 7, 6, 2, 2, 4, 2, 4, 1, 6, 7, 7, 7, 1, 5, 3, 6, 7, 7, 6, 2, 4, 4, 0, 2, 6, 1, 3, 0, 1, 1, 5, 6, 3, 6, 1, 1, 4, 2, 3, 6, 2, 3, 2, 6, 5, 4, 7, 4, 6, 2, 2, 6, 5, 4, 0, 3, 0, 4, 4, 1, 1, 1, 4, 3, 2, 6, 6, 1, 4, 2, 4, 6, 5, 6, 0, 2, 4, 3, 6, 2, 6, 2, 3, 3, 7, 6, 2, 7, 6, 1, 5, 7, 7, 7, 1, 2, 1, 7, 1, 2, 0, 6, 6, 4, 1, 4, 5, 4, 0, 3, 4, 3, 4, 7, 0, 1, 7, 5, 4, 4, 2, 5]


In [45]:
final_predicted = []
for i in range(0, len(Y_pred)):
    if (predicted[i] == 0):
        final_predicted.append("manipuri")
    elif (predicted[i] == 1):
        final_predicted.append("bharatanatyam")
    elif (predicted[i] == 2):
        final_predicted.append("odissi")
    elif (predicted[i] == 3):
        final_predicted.append("kathakali")
    elif (predicted[i] == 4):
        final_predicted.append("kathak")
    elif (predicted[i] == 5):
        final_predicted.append("sattriya")
    elif (predicted[i] == 6):
        final_predicted.append("kuchipudi")
    else:
        final_predicted.append("mohiniyattam")

In [46]:
print(final_predicted)

['kuchipudi', 'mohiniyattam', 'odissi', 'kuchipudi', 'bharatanatyam', 'kuchipudi', 'bharatanatyam', 'kathakali', 'sattriya', 'bharatanatyam', 'mohiniyattam', 'bharatanatyam', 'sattriya', 'sattriya', 'sattriya', 'bharatanatyam', 'odissi', 'kathak', 'mohiniyattam', 'bharatanatyam', 'bharatanatyam', 'kathakali', 'kuchipudi', 'odissi', 'mohiniyattam', 'kathakali', 'odissi', 'bharatanatyam', 'bharatanatyam', 'bharatanatyam', 'mohiniyattam', 'kuchipudi', 'kathak', 'bharatanatyam', 'mohiniyattam', 'kuchipudi', 'odissi', 'odissi', 'kathak', 'odissi', 'kathak', 'bharatanatyam', 'kuchipudi', 'mohiniyattam', 'mohiniyattam', 'mohiniyattam', 'bharatanatyam', 'sattriya', 'kathakali', 'kuchipudi', 'mohiniyattam', 'mohiniyattam', 'kuchipudi', 'odissi', 'kathak', 'kathak', 'manipuri', 'odissi', 'kuchipudi', 'bharatanatyam', 'kathakali', 'manipuri', 'bharatanatyam', 'bharatanatyam', 'sattriya', 'kuchipudi', 'kathakali', 'kuchipudi', 'bharatanatyam', 'bharatanatyam', 'kathak', 'odissi', 'kathakali', 'kuc

In [47]:
pd.DataFrame(list(zip(test_set.Image, final_predicted)),columns=["Image","target"]).to_csv("submission.csv",index=None)